!tar -xvzf yamnet-tensorflow2-yamnet-v1.tar.gz

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import librosa  # For easy audio loading
from IPython.display import Audio

# Verify the model folder path (usually it extracts to the current dir or a subfolder)
model_path = './' # Change if it extracted to a subfolder like 'yamnet_v1'


In [10]:
# Load the model into memory
model = hub.load(model_path)

# Load the class names from your uploaded CSV
class_map_path = "assets/yamnet_class_map.csv"
class_names = pd.read_csv(class_map_path)['display_name'].tolist()

print(f"Model loaded. It can recognize {len(class_names)} sounds.")
# For Sentiguard, check these specifically:
print(f"Class 42: {class_names[42]}") # Cough
print(f"Class 79: {class_names[79]}") # Hiss


Model loaded. It can recognize 521 sounds.
Class 42: Cough
Class 79: Hiss


In [11]:
def classify_audio(audio_file, top_k=5):
    # Load and resample to 16kHz
    waveform, sr = librosa.load(audio_file, sr=16000, mono=True)

    # Run the model
    # YAMNet model expects a 1D float32 waveform at 16kHz
    scores, embeddings, spectrogram = model(tf.constant(waveform, dtype=tf.float32))

    # Get the top predictions
    mean_scores = np.mean(scores, axis=0)
    top_k_indices = np.argsort(mean_scores)[::-1][:top_k] # Get indices of top k scores
    top_k_predictions = [class_names[i] for i in top_k_indices]
    top_k_scores = [mean_scores[i] for i in top_k_indices]

    return top_k_predictions, top_k_scores

# New test cell - please uncomment and replace 'your_audio_file.wav' with your uploaded file names.
# For example:
# top_predictions, top_scores = classify_audio('test_hiss.wav')
# print(f"Detected top {len(top_predictions)} predictions:")
# for i in range(len(top_predictions)):
#     print(f"- {top_predictions[i]} ({top_scores[i]*100:.2f}%) ")

# top_predictions_2, top_scores_2 = classify_audio('test_cough.wav')
# print(f"\nDetected top {len(top_predictions_2)} predictions for another file:")
# for i in range(len(top_predictions_2)):
#     print(f"- {top_predictions_2[i]} ({top_scores_2[i]*100:.2f}%) ")

In [12]:
def sentiguard_monitor(audio_file):
    top_predictions, top_scores = classify_audio(audio_file)

    print(f"Detected top sounds: {top_predictions}")

    # Logic for your 'Active Guardian' Pillar
    alert_triggered = False
    for sound in top_predictions:
        if sound in ["Cough", "Hiss", "Screaming"]:
            print(f"⚠️ ALERT: {sound} detected! Triggering Vibration & Logging Evidence...")
            # In the Android app, this is where Pillar 3 (Logging) would start.
            alert_triggered = True
            break # Trigger alert for the first matching sound

    if not alert_triggered:
        print("Safe environment.")

### Classify your own audio file

To use this, upload your audio file to Colab (e.g., drag and drop it into the files tab on the left sidebar). Then, replace `'your_audio_file.wav'` in the code below with the name of your uploaded file.

In [13]:
# Replace 'your_audio_file.wav' with the actual name of your audio file
# For example, if you uploaded 'my_test_sound.mp3', change it to classify_audio('my_test_sound.mp3')

user_audio_file = 'audio_file.mp3' # <--- Change this line

try:
    top_predictions, top_scores = classify_audio(user_audio_file)
    print(f"\nClassification results for '{user_audio_file}':")
    for i in range(len(top_predictions)):
        print(f"- {top_predictions[i]} ({top_scores[i]*100:.2f}%) ")
except FileNotFoundError:
    print(f"Error: The file '{user_audio_file}' was not found. Please make sure you have uploaded the file and the filename is correct.")
except Exception as e:
    print(f"An error occurred during classification: {e}")


Classification results for 'audio_file.mp3':
- Cough (43.66%) 
- Throat clearing (25.39%) 
- Silence (23.54%) 
- Speech (11.58%) 
- Beatboxing (5.13%) 


In [14]:
# Install pydub for more robust audio file handling, especially for .mp3
!pip install pydub

In [15]:
import tensorflow as tf

# 1. Path to your extracted SavedModel directory
# Ensure you have run !tar -xvzf yamnet-tensorflow2-yamnet-v1.tar.gz first
saved_model_dir = './'

# 2. Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Optional: Add quantization to shrink the size for better Edge performance
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# 3. Save as your preferred filename
with open('sound_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

print("Conversion complete: sound_classifier.tflite is ready.")


Conversion complete: sound_classifier.tflite is ready.
